In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [2]:
df_pib = pd.read_csv("https://raw.githubusercontent.com/afonsosr2/dataviz-graficos-composicao-relacionamento/master/dados/pib_br_2002_2020_estados.csv")
df_pib

,ano,sigla_uf,regiao,pib,impostos_liquidos,va,va_agropecuaria,va_industria,va_servicos,va_adespss
0,2002,RO,Norte,7467629886,839731192,6627898698,715526872,1191090432,2484579193,2236702207
1,2003,RO,Norte,9425010486,1108434936,8316575548,1278658831,1216605061,3376727040,2444584625
2,2004,RO,Norte,11004641436,1288806654,9715834778,1288515348,1674933817,3986529419,2765856199
3,2005,RO,Norte,12511821181,1476144194,11035676990,1342222120,1887932121,4603783904,3201738843
4,2006,RO,Norte,13054713344,1613809974,11440903374,1238006193,2210692147,4320526746,3671678293
...,...,...,...,...,...,...,...,...,...,...
508,2016,DF,Centro-Oeste,235540044811,29145619376,206394425435,820754661,9662357225,103859865830,92051447720
509,2017,DF,Centro-Oeste,244722249337,29120461647,215601787690,828313642,8448768236,108322119432,98002586380
510,2018,DF,Centro-Oeste,254817204692,28692287369,226124917323,1022690641,9541298290,113768086938,101792841454
511,2019,DF,Centro-Oeste,273613711477,30686607647,242927103829,992393584,9453608031,125261853488,107219248727


<font color="#66ff00" size=6 > Gráfico de Cascata </font>

Como está distribuído o PIB do Estado da Bahia no ano de 2020, separado por impostos líquidos e valores adicionados brutos.

In [3]:
df_pib.query('sigla_uf == "BA" and ano == 2020')

,ano,sigla_uf,regiao,pib,impostos_liquidos,va,va_agropecuaria,va_industria,va_servicos,va_adespss
303,2020,BA,Nordeste,305320812691,37094030606,268226782074,28006935845,59491629932,122722534188,58005682122


In [4]:
# Transformando a consulta acima em um DataFrame.
df_bahia = df_pib.query('sigla_uf == "BA" and ano == 2020')[["pib", "impostos_liquidos","va"]]
# df_bahia.reset_index(drop=True).T


In [5]:
df_bahia = df_bahia.rename(
    columns={
        "pib": "PIB",
        "impostos_liquidos": "Impostos Líquidos",
        "va": "Valor Adicionado",
    }
)
df_bahia

,PIB,Impostos Líquidos,Valor Adicionado
303,305320812691,37094030606,268226782074


In [6]:
df_indicadores_BA = pd.melt(df_bahia, var_name="Indicadores", value_name="Valores")
df_indicadores_BA

,Indicadores,Valores
0,PIB,305320812691
1,Impostos Líquidos,37094030606
2,Valor Adicionado,268226782074


In [7]:
# Trocar o valor dos impostos para negativo
mask_impostos = (df_indicadores_BA["Indicadores"] == "Impostos Líquidos")
df_indicadores_BA.loc[mask_impostos, "Valores"] *= -1
display(df_indicadores_BA)

,Indicadores,Valores
0,PIB,305320812691
1,Impostos Líquidos,-37094030606
2,Valor Adicionado,268226782074


In [8]:
df_indicadores_BA

,Indicadores,Valores
0,PIB,305320812691
1,Impostos Líquidos,-37094030606
2,Valor Adicionado,268226782074


In [9]:
%%timeit
df_indicadores_BA["Valores_str_apply"] = df_indicadores_BA["Valores"].apply(lambda x: f"R$ {x/(1E9):,.2f} Bi")
# Meu método

66.4 μs ± 4.75 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
%%timeit
df_indicadores_BA["Valores_str_map"] = (df_indicadores_BA["Valores"] / 1e9).map(
    "R$ {:,.2f} Bi".format
)
# Método da aula.

89 μs ± 1.4 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
# Revertendo o mini benchmark
df_indicadores_BA = df_indicadores_BA.drop(columns="Valores_str_map").rename(columns={"Valores_str_apply":"Valores_str"})
df_indicadores_BA["Medidas"] = ["absolute", "relative", "total"]

In [12]:
chart_cascata = go.Waterfall(
    name="", orientation="v",
    measure=df_indicadores_BA["Medidas"],
    x=df_indicadores_BA.Indicadores, y=df_indicadores_BA.Valores,
    text=df_indicadores_BA["Valores_str"], textposition="outside",
    connector_line=dict(color="lightgray"),
    totals_marker=dict(color="#1E0FC3"),
    decreasing_marker=dict(color="#FF0000")
)

fig = go.Figure(chart_cascata)

TITLE = (
    "Distribuição do Produto Interno Bruto (PIB) da Bahia em 2020"
    "<br><sup style='color:#555655'>"
    "Impostos líquidos e Valores Adicionados Brutos de bens e serviços "
    "(em bilhões de reais)</sup>"
)
layout = dict(
    width=1000,
    height=500,
    plot_bgcolor="white",
    hovermode="closest",
    font=dict(family="DejaVu Sans", size=15, color="#636363"),
    title=dict(
        text=TITLE, font=dict(size=24, color="#7C7C7C"),pad=dict(l=50),
    ),
    yaxis=dict(range=[0, 380e9]),
)

fig.update_layout(**layout)
fig.update_yaxes(showticklabels=False) # 
fig.update_traces(hovertemplate="<b>%{x}</b> = %{text}")


# --- 
# Fonte dos Dados
fontedados_HTML = 'Fonte dos dados: <a href="https://sidra.ibge.gov.br/pesquisa/pib-munic/tabelas">Produto Interno Bruto | IBGE</a>'
fig.add_annotation(text=fontedados_HTML, align="left", xref="paper", yref = "paper", x=1, y=-0.2, showarrow=False)
fig.show()

---



**Qual a evolução anual do PIB do Rio de Janeiro entre os anos de 2010 a 2020?**

In [13]:
anos = [ano for ano in range(2010,2021)]
display(df_pib.query('sigla_uf == "RJ" and ano == @anos').reset_index(drop=True)[["ano", "pib"]])

# Armazenando a query em um novo dataframe que acompanha a evolução anual do pib RJ
df_rio = df_pib.query('sigla_uf == "RJ" and ano == @anos').reset_index(drop=True)[["ano", "pib"]]

,ano,pib
0,2010,449858101109
1,2011,512767904769
2,2012,574884973130
3,2013,628226069362
4,2014,671076844311
5,2015,659138951833
6,2016,640401206447
7,2017,671605668055
8,2018,758859046865
9,2019,779927917084


In [14]:
# Estabelece o pib inicial ↓ como o "nível"
df_rio["var_pib"] = df_rio.pib.diff().fillna(value=df_rio.pib).astype("int64")
df_rio["medidas"] = ["absolute"] + 10 * ["relative"]
df_rio

,ano,pib,var_pib,medidas
0,2010,449858101109,449858101109,absolute
1,2011,512767904769,62909803660,relative
2,2012,574884973130,62117068361,relative
3,2013,628226069362,53341096232,relative
4,2014,671076844311,42850774949,relative
5,2015,659138951833,-11937892478,relative
6,2016,640401206447,-18737745386,relative
7,2017,671605668055,31204461608,relative
8,2018,758859046865,87253378810,relative
9,2019,779927917084,21068870219,relative


In [15]:
# Concatenar uma última linha com o Total
temp_df = pd.Series(
    {
        "ano": "Total",
        "pib": df_rio.pib.values[-1],
        "var_pib": df_rio.var_pib.values[-1],
        "medidas": "total"
    }
).to_frame()

print("pd.Series => DataFrame")
display(temp_df)

print("O DataFrame transposto como última linha")
display(temp_df.T)

print("Adicionando o registro ao df_rio (formatado)")
df_rio = pd.concat([df_rio, temp_df.T], axis=0, ignore_index=True)
# df_rio = df_rio.drop(columns=["pib"]) # Não mais necesária
df_rio["var_pib_text"] = (df_rio["var_pib"].apply(lambda x: f"R$ {(x/1E9):,.2f} Bi"))

# Renomear para nomes mais verbosos.
df_rio = df_rio.rename(columns = {"ano":"Ano", "var_pib": "Variação do PIB", "var_pib_text":"Variação do PIB (em texto)"})
display(df_rio)

del temp_df

pd.Series => DataFrame


,0
ano,Total
pib,753823710636
var_pib,-26104206448
medidas,total


O DataFrame transposto como última linha


,ano,pib,var_pib,medidas
0,Total,753823710636,-26104206448,total


Adicionando o registro ao df_rio (formatado)


,Ano,pib,Variação do PIB,medidas,Variação do PIB (em texto)
0,2010,449858101109,449858101109,absolute,R$ 449.86 Bi
1,2011,512767904769,62909803660,relative,R$ 62.91 Bi
2,2012,574884973130,62117068361,relative,R$ 62.12 Bi
3,2013,628226069362,53341096232,relative,R$ 53.34 Bi
4,2014,671076844311,42850774949,relative,R$ 42.85 Bi
5,2015,659138951833,-11937892478,relative,R$ -11.94 Bi
6,2016,640401206447,-18737745386,relative,R$ -18.74 Bi
7,2017,671605668055,31204461608,relative,R$ 31.20 Bi
8,2018,758859046865,87253378810,relative,R$ 87.25 Bi
9,2019,779927917084,21068870219,relative,R$ 21.07 Bi


Aqui vou copiar tudo da aula porque achei ela MUITO RUIM.

In [16]:
CINZA1, CINZA2, CINZA3, CINZA4, CINZA5, BRANCO = '#231F20', '#414040', '#555655', '#A6A6A5', '#BFBEBE', '#FFFFFF'
AZUL1, AZUL2, AZUL3, AZUL4, AZUL5, AZUL6 = '#174A7E', '#4A81BF', "#6495ED", '#2596BE', '#94AFC5', '#CDDBF3'
VERMELHO1, VERMELHO2, LARANJA1 = '#C3514E',	'#E6BAB7',	'#F79747'
VERDE1, VERDE2, VERDE3 = '#0C8040',	'#9ABB59', '#9ECCB3'

fig = go.Figure(
                go.Waterfall(name = "", orientation = "v", measure = df_rio["medidas"],
                             y = df_rio["Variação do PIB"], text = df_rio["Variação do PIB (em texto)"],
                             textposition = "outside",
                             connector_line = dict(color = CINZA3),
                             totals_marker = dict(color = AZUL3),
                             increasing_marker = dict(color = VERDE1),
                             decreasing_marker = dict(color = VERMELHO1)
                             )
                )

# Personalizando o gráfico
fig.update_layout(width=1300, height=600, font_family = 'DejaVu Sans', font_size=15,
                  font_color= CINZA2, title_font_color= CINZA1, title_font_size=24,
                  title_text='Variação do Produto Interno Bruto (PIB) do Rio de Janeiro' +
                             'De 2010 a 2020 (em bilhões de reais)',
                  plot_bgcolor= BRANCO, yaxis_range=[0,850e9])

# Retirando os ticks do eixo y
fig.update_yaxes(showticklabels=False)

# Ajustando o eixo x para receber o Total
fig.update_xaxes(tickmode='array', tickvals=np.arange(0,12), ticktext=df_rio["Ano"])

# Dados ao passar o mouse
fig.update_traces(hovertemplate = "%{x} = %{text}")

fig.add_annotation(text="O Rio de Janeiro apresentou recuo no PIBem pelo menos 3 anos no período:2015, 2016 e 2020",
                   align="left", axref = 'x', ayref='y', x=5, y=600e9, ax=8, ay=450e9,
                   arrowhead=1, arrowside = "start", showarrow=True, arrowwidth=2, arrowcolor=CINZA3,
                   bordercolor= CINZA3, borderwidth=1, borderpad=4)
fig.add_annotation(text="", axref = 'x', ayref='y', x=6, y=600e9, ax=8, ay=530e9,
                   arrowhead=1, arrowside = "start", showarrow=True, arrowwidth=2, arrowcolor=CINZA3)
fig.add_annotation(text="", axref = 'x', ayref='y', x=10, y=710e9, ax=8.5, ay=530e9,
                   arrowhead=1, arrowside = "start", showarrow=True, arrowwidth=2, arrowcolor=CINZA3)

fig.show()